# Getting Data

In [1]:
from decouple import Config,RepositoryEnv
env_path = '/home/sfniila/.ssh/.env'
env_config = Config(RepositoryEnv(env_path))
github_token = env_config.get('GITHUB_TOKEN')

## PyGitHub

In [2]:
from github import Github

def pygithub_get_repo_paths(
    token: str,
    owner: str, 
    name: str
) -> any:
    g = Github(token)
    repo = g.get_repo(f"{owner}/{name}")
    contents = repo.get_contents("")
    paths = []
    while len(contents) > 0:
      file_content = contents.pop(0)
      if file_content.type == 'dir':
        contents.extend(repo.get_contents(file_content.path))
      else:
        paths.append(file_content.path)
    g.close()
    return paths

def pygithub_get_path_content(
    token: str,
    owner: str, 
    name: str, 
    path: str
) -> any:
    g = Github(token)
    repo = g.get_repo(f"{owner}/{name}")
    file_content = repo.get_contents(path)
    content = file_content.decoded_content.decode('utf-8')
    g.close()
    return content

## Scraping Functions

In [49]:
import os

def get_github_repo_documents(
    github_token: str,
    repo_owner: str,
    repo_name: str,
    relevant_files: any,
    store: bool
) -> any:
    storage_path = os.getcwd() + '/' + repo_owner + '-' + repo_name + '.txt' 

    repo_paths = []
    if not os.path.exists(storage_path):
        print('Getting github paths')
        repo_paths = pygithub_get_repo_paths(
            token = github_token,
            owner = repo_owner, 
            name = repo_name
        )
        
        if store:
            print('Storing paths')
            path_amount = len(repo_paths)
            i = 0
            with open(storage_path, 'w') as file:
                for line in repo_paths:
                    row = line 
                    if i < path_amount:
                        row += '\n'
                    file.write(row)
                    i += 1
            print('Paths stored')
    else:
        print('Getting stored paths')
        with open(storage_path, 'r') as file:
            repo_paths = file.readlines()
    print('Paths fetched')
                
    print('Filtering paths')
    relevant_paths = []
    for path in repo_paths:
        path_split = path.split('/')
        file_end = path_split[-1].split('.')[-1].rstrip()
        if file_end in relevant_files:
            relevant_paths.append(path.rstrip())
    print('Paths filtered')

    formatted_paths = {
        'paths': relevant_paths
    }

    # consider how to store this to allas
    return formatted_paths

In [4]:
import requests
import json
import time

def get_document_data(
    document_url: str,
    document_type: str
) -> any:
    data = None
    response = requests.get(
        url = document_url
    )
    if response.status_code == 200:
        if document_type == 'text':
            data = response.text
        if document_type == 'json':
            data = json.loads(response.text)
        # handle html later
    return data

def scrape_documents(
    url_list: any,
    timeout: int
) -> any:
    documents = []

    text_files = [
        'py',
        'md',
        'yaml',
        'sh'
    ]

    json_files = [
        'ipynb'
    ]
    index = 0
    for url in url_list:
        document = {
            'name': '',
            'data': ''
        }
        url_split = url.split('/')
        if 'github' in url_split[2]:
            if 'raw' in url_split[2]:
                file_end = url_split[-1].split('.')[-1]
                document['name'] = url_split[-1]
                if file_end in text_files:
                    document['data'] = get_document_data(
                        document_url = url,
                        document_type = 'text' 
                    )
                if file_end in json_files:
                    document['data'] = get_document_data(
                        document_url = url,
                        document_type = 'json' 
                    )
        documents.append(document)
        index = index + 1
        if index < len(url_list):
            time.sleep(timeout)
    return documents

Files of intrest
- md = important to usesr
- txt = isn't important to users
- sh = isn't important to users
- yaml = important to developers
- py = important to users and developers
- ipynb = important to users and developers

In [50]:
repository_paths = get_github_repo_documents(
    github_token = github_token,
    repo_owner = 'K123AsJ0k1',
    repo_name = 'cloud-hpc-oss-mlops-platform',
    relevant_files = [
        'md',
        'yaml',
        'py',
        'ipynb'
    ],
    store = True
)

Getting stored paths
Paths fetched
Filtering paths
Paths filtered


# Storing Data

## TreeSitter Functions

In [6]:
import tree_sitter_python as tspython
from tree_sitter import Language, Parser
import re

def tree_extract_imports(
    node: any, 
    code_text: str
) -> any:
    imports = []
    if node.type == 'import_statement' or node.type == 'import_from_statement':
        start_byte = node.start_byte
        end_byte = node.end_byte
        imports.append(code_text[start_byte:end_byte].decode('utf8'))
    for child in node.children:
        imports.extend(tree_extract_imports(child, code_text))
    return imports

def tree_extract_dependencies(
    node: any, 
    code_text: str
) -> any:
    dependencies = []
    for child in node.children:
        if child.type == 'call':
            dependency_name = child.child_by_field_name('function').text.decode('utf8')
            dependencies.append(dependency_name)
        dependencies.extend(tree_extract_dependencies(child, code_text))
    return dependencies

def tree_extract_code_and_dependencies(
    node: any,
    code_text: str
) -> any:
    codes = []
    if not node.type == 'function_definition':
        start_byte = node.start_byte
        end_byte = node.end_byte
        name = node.child_by_field_name('name')
        if name is None:
            code = code_text[start_byte:end_byte].decode('utf8')
            if not 'def' in code:
                dependencies = tree_extract_dependencies(node, code_text)
                codes.append({
                    'name': 'global',
                    'code': code,
                    'dependencies': dependencies
                })
    return codes

def tree_extract_functions_and_dependencies(
    node: any, 
    code_text: str
) -> any:
    functions = []
    if node.type == 'function_definition':
        start_byte = node.start_byte
        end_byte = node.end_byte
        name = node.child_by_field_name('name').text.decode('utf8')
        code = code_text[start_byte:end_byte].decode('utf8')
        dependencies = tree_extract_dependencies(node, code_text)
        functions.append({
            'name': name,
            'code': code,
            'dependencies': dependencies
        })
    for child in node.children:
        functions.extend(tree_extract_functions_and_dependencies(child, code_text))
    return functions

def tree_get_used_imports(
    general_imports: any,
    function_dependencies: any
) -> any:
    parsed_imports = {}
    for code_import in general_imports:
        import_factors = code_import.split('import')[-1].replace(' ', '')
        import_factors = import_factors.split(',')
    
        for factor in import_factors:
            if not factor in parsed_imports:
                parsed_imports[factor] = code_import.split('import')[0] + 'import ' + factor
            
    relevant_imports = {}
    for dependency in function_dependencies:
        initial_term = dependency.split('.')[0]
    
        if not initial_term in relevant_imports:
            if initial_term in parsed_imports:
                relevant_imports[initial_term] = parsed_imports[initial_term]
    
    used_imports = []
    for name, code in relevant_imports.items():
        used_imports.append(code)

    return used_imports

def tree_get_used_functions(
    general_functions: any,
    function_dependencies: any
): 
    used_functions = []
    for related_function_name in function_dependencies:
        for function in general_functions:
            if function['name'] == related_function_name:
                used_functions.append('from ice import ' + function['name'])
    return used_functions

def tree_create_code_document(
    code_imports: any,
    code_functions: any,
    function_item: any
) -> any:
    used_imports = tree_get_used_imports(
        general_imports = code_imports,
        function_dependencies = function_item['dependencies']
    )

    used_functions = tree_get_used_functions(
        general_functions = code_functions,
        function_dependencies = function_item['dependencies']
    )
    
    document = {
        'imports': used_imports,
        'functions': used_functions,
        'name': function_item['name'],
        'dependencies': function_item['dependencies'],
        'code': function_item['code']
    }
    
    return document
     
def tree_format_code_document(
    code_document: any
) -> any:
    formatted_document = ''
    for doc_import in code_document['imports']:
        formatted_document += doc_import + '\n'

    for doc_functions in code_document['functions']:
        formatted_document += doc_functions + '\n'

    if 0 < len(code_document['dependencies']):
        formatted_document += 'code dependencies\n'

        for doc_dependency in code_document['dependencies']:
            formatted_document += doc_dependency + '\n'

    if code_document['name'] == 'global':
        formatted_document += code_document['name'] + ' code\n'
    else:
        formatted_document += 'function ' + code_document['name'] + ' code\n'
    
    for line in code_document['code'].splitlines():
        if not bool(line.strip()):
            continue
        doc_code = re.sub(r'#.*','', line)
        if not bool(doc_code.strip()):
            continue
        formatted_document += doc_code + '\n'    
    return formatted_document

def tree_create_python_code_and_function_documents(
    code_document: any
):
    PY_LANGUAGE = Language(tspython.language())
    parser = Parser(PY_LANGUAGE)
   
    tree = parser.parse(
        bytes(
            code_document,
            "utf8"
        )
    )

    root_node = tree.root_node
    code_imports = tree_extract_imports(
        root_node, 
        bytes(
            code_document, 
            'utf8'
        )
    )

    code_global = tree_extract_code_and_dependencies(
        root_node, 
        bytes(
            code_document, 
            'utf8'
        )
    )

    code_functions = tree_extract_functions_and_dependencies(
        root_node, 
        bytes(
            code_document, 
            'utf8'
        )
    )
    
    initial_documents = []
    for item in code_global:
        document = tree_create_code_document(
            code_imports = code_imports,
            code_functions = code_functions,
            function_item = item
        )  
        initial_documents.append(document)

    for item in code_functions:
        document = tree_create_code_document(
            code_imports = code_imports,
            code_functions = code_functions,
            function_item = item
        )  
        initial_documents.append(document)

    formatted_documents = []
    seen_functions = []
    for document in initial_documents:
        if not document['name'] == 'global':
            if document['name'] in seen_functions:
                continue
        
        formatted_document = tree_format_code_document(
            code_document = document
        )

        formatted_documents.append(formatted_document)
        seen_functions.append(document['name'])
    return formatted_documents

## Document Functions

In [7]:
import nbformat
from bs4 import BeautifulSoup
import markdown

def extract_jupyter_notebook_markdown_and_code(
    notebook_document: any
): 
    notebook_documents = {
        'markdown': [],
        'code': []
    }

    notebook = nbformat.from_dict(notebook_document)

    index = 1
    for cell in notebook.cells:
        if cell.cell_type == 'markdown':
            notebook_documents['markdown'].append({
                'id': index,
                'data': cell.source
            })
            index += 1
        if cell.cell_type == 'code':
            notebook_documents['code'].append({
                'id': index,
                'data': cell.source
            })
            index += 1
    
    return notebook_documents
    
def parse_markdown_into_text(
    markdown_text: any
) -> any:
    html = markdown.markdown(markdown_text)
    soup = BeautifulSoup(html, features='html.parser')
    text = soup.get_text()
    code_block_pattern = re.compile(r"```")
    text = re.sub(code_block_pattern, '', text)
    text = text.rstrip('\n')
    text = text.replace('\nsh', '\n')
    text = text.replace('\nbash', '\n')
    return text

def create_python_documents(
    python_document: any
): 
    joined_code = ''.join(python_document)
    block_code_documents = tree_create_python_code_and_function_documents(
        code_document = joined_code
    )

    code_documents = []
    seen_function_names = []
    code_doc_index = 0
    for code_doc in block_code_documents:
        row_split = code_doc.split('\n')
        for row in row_split:
            if 'function' in row and 'code' in row:
                function_name = row.split(' ')[1]
                if not function_name in seen_function_names:
                    seen_function_names.append(function_name)
                else:
                    del block_code_documents[code_doc_index]
        code_doc_index += 1

    if 0 < len(block_code_documents):
        index = 1
        for code_doc in block_code_documents:
            code_documents.append({
                'index': index,
                'data': code_doc
            })
            index += 1
        
    formatted_documents = {
        'code': code_documents
    }
    return formatted_documents

def create_notebook_documents(
    notebook_document: any
):
    notebook_documents = extract_jupyter_notebook_markdown_and_code(
        notebook_document = notebook_document
    )

    markdown_documents = []
    for block in notebook_documents['markdown']:
        joined_text = ''.join(block['data'])
        markdown_text = parse_markdown_into_text(
            markdown_text = joined_text
        )
        markdown_documents.append({
            'index': block['id'],
            'data': markdown_text
        })
        
    code_documents = []
    seen_function_names = []
    for block in notebook_documents['code']:
        joined_code = ''.join(block['data'])
        block_code_documents = tree_create_python_code_and_function_documents(
            code_document = joined_code
        )

        code_doc_index = 0
        for code_doc in block_code_documents:
            row_split = code_doc.split('\n')
            for row in row_split:
                if 'function' in row and 'code' in row:
                    function_name = row.split(' ')[1]
                    if not function_name in seen_function_names:
                        seen_function_names.append(function_name)
                    else:
                        del block_code_documents[code_doc_index]
            code_doc_index += 1
        
        if 0 < len(block_code_documents):
            sub_indexes = False
            if 1 < len(block_code_documents):
                sub_indexes = True
            index = 1
            for code_doc in block_code_documents:
                if sub_indexes:
                    code_documents.append({
                        'sub-index': index, 
                        'index': block['id'],
                        'data': code_doc
                    })
                else:
                    code_documents.append({ 
                        'index': block['id'],
                        'data': code_doc
                    })
                index += 1
            
    formatted_documents = {
        'markdown': markdown_documents,
        'code': code_documents
    }
    
    return formatted_documents

## Mongo Functions

In [8]:
from pymongo import MongoClient as mc

def mongo_is_client(
    storage_client: any
) -> any:
    return isinstance(storage_client, mc.Connection)

def mongo_setup_client(
    username: str,
    password: str,
    address: str,
    port: str
) -> any:
    connection_prefix = 'mongodb://(username):(password)@(address):(port)/'
    connection_address = connection_prefix.replace('(username)', username)
    connection_address = connection_address.replace('(password)', password)
    connection_address = connection_address.replace('(address)', address)
    connection_address = connection_address.replace('(port)', port)
    mongo_client = mc(
        host = connection_address
    )
    return mongo_client

def mongo_get_database(
    mongo_client: any,
    database_name: str
) -> any:
    try:
        database = mongo_client[database_name]
        return database
    except Exception as e:
        return None

def mongo_check_database(
    mongo_client: any, 
    database_name: str
) -> bool:
    try:
        database_exists = database_name in mongo_client.list_database_names()
        return database_exists
    except Exception as e:
        return False

def mongo_list_databases(
    mongo_client: any
) -> any:
    try:
        databases = mongo_client.list_database_names()
        return databases
    except Exception as e:
        return []

def mongo_remove_database(
    mongo_client: any, 
    database_name: str
) -> bool:
    try:
        mongo_client.drop_database(database_name)
        return True
    except Exception as e:
        return False

def mongo_get_collection(
    mongo_client: any, 
    database_name: str, 
    collection_name: str
) -> bool:
    try:
        database = mongo_get_database(
            mongo_client = mongo_client,
            database_name = database_name
        )
        collection = database[collection_name]
        return collection
    except Exception as e:
        return None
    
def mongo_check_collection(
    mongo_client: any, 
    database_name: any, 
    collection_name: any
) -> bool:
    try:
        database = mongo_client[database_name]
        collection_exists = collection_name in database.list_collection_names()
        return collection_exists
    except Exception as e:
        return False

def mongo_update_collection(
    mongo_client: any, 
    database_name: str, 
    collection_name: str, 
    filter_query: any, 
    update_query: any
) -> any:
    try:
        collection = mongo_get_collection(
            mongo_client = mongo_client, 
            database_name = database_name, 
            collection_name = collection_name
        )
        result = collection.update_many(filter_query, update_query)
        return result
    except Exception as e:
        return None

def mongo_list_collections(
    mongo_client: any, 
    database_name: str
) -> bool:
    try:
        database = mongo_get_database(
            mongo_client = mongo_client,
            database_name = database_name
        )
        collections = database.list_collection_names()
        return collections
    except Exception as e:
        return []

def mongo_remove_collection(
    mongo_client: any, 
    database_name: str, 
    collection_name: str
) -> bool:
    try: 
        database = mongo_get_database(
            mongo_client = mongo_client,
            database_name = database_name
        )
        database.drop_collection(collection_name)
        return True
    except Exception as e:
        return False

def mongo_create_document(
    mongo_client: any, 
    database_name: str, 
    collection_name: str, 
    document: any
) -> any:
    try: 
        collection = mongo_get_collection(
            mongo_client = mongo_client, 
            database_name = database_name, 
            collection_name = collection_name
        )
        result = collection.insert_one(document)
        return result
    except Exception as e:
        return None

def mongo_get_document(
    mongo_client: any, 
    database_name: str, 
    collection_name: str, 
    filter_query: any
):
    try: 
        collection = mongo_get_collection(
            mongo_client = mongo_client, 
            database_name = database_name, 
            collection_name = collection_name
        )
        document = collection.find_one(filter_query)
        return document
    except Exception as e:
        print(e)
        return None 

def mongo_list_documents(
    mongo_client: any, 
    database_name: str, 
    collection_name: str, 
    filter_query: any,
    sorting_query: any
) -> any:
    try: 
        collection = mongo_get_collection(
            mongo_client = mongo_client, 
            database_name = database_name, 
            collection_name = collection_name
        )
        documents = list(collection.find(filter_query).sort(sorting_query))
        return documents
    except Exception as e:
        return []

def mongo_update_document(
    mongo_client: any, 
    database_name: any, 
    collection_name: any, 
    filter_query: any, 
    update_query: any
) -> any:
    try: 
        collection = mongo_get_collection(
            mongo_client = mongo_client, 
            database_name = database_name, 
            collection_name = collection_name
        )
        result = collection.update_one(filter_query, update_query)
        return result
    except Exception as e:
        return None

def mongo_remove_document(
    mongo_client: any, 
    database_name: str, 
    collection_name: str, 
    filter_query: any
) -> bool:
    try: 
        collection = mongo_get_collection(
            mongo_client = mongo_client, 
            database_name = database_name, 
            collection_name = collection_name
        )
        result = collection.delete_one(filter_query)
        return result
    except Exception as e:
        return None

## Storage Functions

In [9]:
mongo_client = mongo_setup_client(
    username = 'mongo123',
    password = 'mongo456',
    address = '127.0.0.1',
    port = '27017'
)

## Markdown documents

In [10]:
import markdown
from bs4 import BeautifulSoup

def create_markdown_documents(
    markdown_text: any
) -> any:
    html = markdown.markdown(markdown_text)
    soup = BeautifulSoup(html, features='html.parser')
    code_block_pattern = re.compile(r"```")
    
    documents = []
    document = ''
    index = 1
    for element in soup.descendants:
        if element.name in ['h2', 'h3', 'h4', 'h5', 'h6']:
            text = element.get_text(strip = True)
            if not document == '':
                document = document.replace('\n', '')
                if not len(document.split()) == 1:
                    documents.append({
                        'index': index,
                        'sub-index': 0,
                        'type': 'markdown',
                        'data': document
                    })
                    index += 1
                document = ''
            document += text
        elif element.name == 'p':
            text = element.get_text(strip = True)
            text = re.sub(code_block_pattern, '', text)
            text = text.rstrip('\n')
            text = text.replace('\nsh', '')
            text = text.replace('\nbash', '')
            document += ' ' + text
        elif element.name in ['ul', 'ol']:
            text = ''
            for li in element.find_all('li'):
                item = li.get_text(strip=True)
                if not '-' in item:
                    text += '-' + item
                    continue
                text += item
            document += ' ' + text
            
    documents.append({
        'index': index,
        'sub-index': 0,
        'type': 'markdown',
        'data': document
    })
    
    formatted_documents = {
        'markdown': documents
    }
    
    return formatted_documents

In [83]:
example_data = pygithub_get_path_content(
    token = github_token,
    owner = 'K123AsJ0k1', 
    name = 'cloud-hpc-oss-mlops-platform', 
    path = repository_paths[7]
)

In [163]:
markdown_documents = create_markdown_documents(
    markdown_text = example_data
) 

## YAML documents

In [58]:
import yaml 

def extract_yaml_values(
    section: any,
    path: str,
    values: any
) -> any:
    for key, value in section.items():
        if path == '':
            current_path = key
        else:
            current_path = path + '/' + key
        if isinstance(value, dict):
            extract_yaml_values(
                section = value,
                path = current_path,
                values = values
            )
        if isinstance(value, list):
            number = 1
            
            for case in value:
                base_path = current_path
                if isinstance(case, dict):
                   extract_yaml_values(
                       section = case,
                       path = current_path,
                       values = values
                   ) 
                   continue
                base_path += '/' + str(number)
                number += 1
                values.append(base_path + '=' + str(case))
        else:
            if isinstance(value, dict):
                continue
            if isinstance(value, list):
                continue
            values.append(current_path + '=' + str(value))
            
    return values

def create_yaml_documents(
    yaml_text: any
) -> any:
    yaml_data = list(yaml.safe_load_all(yaml_text))

    documents = []
    index = 1
    for data in yaml_data:
        yaml_values = extract_yaml_values(
            section = data,
            path = '',
            values = []
        )

        previous_root = ''
        document = ''
        sub_index = 1
        for value in yaml_values:
            equal_split = value.split('=')
            path_split = equal_split[0].split('/')
            root = path_split[0]
            if not root == previous_root:
                if 0 < len(document):
                    documents.append({
                        'index': index,
                        'sub-index': sub_index,
                        'type': 'yaml',
                        'data': document
                    })
                    sub_index += 1
                    
                previous_root = root
                document = value
            else:
                document += value
                
        documents.append({
            'index': index,
            'sub-index': sub_index,
            'type': 'yaml',
            'data': document
        })
        index += 1

    formatted_documents = {
        'yaml': documents
    }
            
    return formatted_documents

In [13]:
repository_paths[18]

'deployment/forwarder/forwarder-namespace.yaml'

In [61]:
example_yaml = pygithub_get_path_content(
    token = github_token,
    owner = 'K123AsJ0k1', 
    name = 'cloud-hpc-oss-mlops-platform', 
    path = 'deployment/forwarder/celery/celery-deployment.yaml'
)

In [ ]:
'deployment/kubeflow/manifests/in-cluster-setup/standalone-kfp/kustomization.yaml'

In [62]:
yaml_documents = create_yaml_documents(
    yaml_text = example_yaml
) 

## Python documents

In [73]:
def create_python_documents(
    python_text: any
): 
    joined_code = ''.join(python_text)
    block_code_documents = tree_create_python_code_and_function_documents(
        code_document = joined_code
    )

    code_documents = []
    seen_function_names = []
    code_doc_index = 0
    for code_doc in block_code_documents:
        row_split = code_doc.split('\n')
        for row in row_split:
            if 'function' in row and 'code' in row:
                function_name = row.split(' ')[1]
                if not function_name in seen_function_names:
                    seen_function_names.append(function_name)
                else:
                    del block_code_documents[code_doc_index]
        code_doc_index += 1

    if 0 < len(block_code_documents):
        index = 1
        for code_doc in block_code_documents:
            code_documents.append({
                'index': index,
                'sub-index': 0,
                'type': 'python',
                'data': code_doc
            })
            index += 1
        
    formatted_documents = {
        'code': code_documents
    }
    return formatted_documents

In [65]:
example_python = pygithub_get_path_content(
    token = github_token,
    owner = 'K123AsJ0k1', 
    name = 'cloud-hpc-oss-mlops-platform', 
    path = 'applications/article/submitter/backend/functions/platforms/flower.py'
)

In [74]:
python_documents = create_python_documents(
    python_text = example_python
)

## Notebook documents

In [96]:
import nbformat

def parse_jupyter_notebook_markdown_into_text(
    markdown_text: any
) -> any:
    html = markdown.markdown(markdown_text)
    soup = BeautifulSoup(html, features='html.parser')
    text = soup.get_text()
    code_block_pattern = re.compile(r"```")
    text = re.sub(code_block_pattern, '', text)
    text = text.rstrip('\n')
    text = text.replace('\nsh', '\n')
    text = text.replace('\nbash', '\n')
    return text

def extract_jupyter_notebook_markdown_and_code(
    notebook_text: any
): 
    notebook_documents = {
        'markdown': [],
        'code': []
    }

    notebook = nbformat.reads(notebook_text, as_version=2)
    index = 1
    for cell in notebook.worksheets[0].cells:
        if cell.cell_type == 'markdown':
            notebook_documents['markdown'].append({
                'id': index,
                'data': cell.source
            })
            index += 1
        if cell.cell_type == 'code':
            notebook_documents['code'].append({
                'id': index,
                'data': cell.input
            })
            index += 1
    
    return notebook_documents

def create_notebook_documents(
    notebook_text: any
):
    notebook_documents = extract_jupyter_notebook_markdown_and_code(
        notebook_text = notebook_text
    )

    markdown_documents = []
    for block in notebook_documents['markdown']:
        joined_text = ''.join(block['data'])
        markdown_text = parse_jupyter_notebook_markdown_into_text(
            markdown_text = joined_text
        )
        markdown_documents.append({
            'index': block['id'],
            'sub-index': 0,
            'type': 'markdown',
            'data': markdown_text
        })
        
    code_documents = []
    seen_function_names = []
    for block in notebook_documents['code']:
        joined_code = ''.join(block['data'])
        block_code_documents = tree_create_python_code_and_function_documents(
            code_document = joined_code
        )

        code_doc_index = 0
        for code_doc in block_code_documents:
            row_split = code_doc.split('\n')
            for row in row_split:
                if 'function' in row and 'code' in row:
                    function_name = row.split(' ')[1]
                    if not function_name in seen_function_names:
                        seen_function_names.append(function_name)
                    else:
                        del block_code_documents[code_doc_index]
            code_doc_index += 1
        
        if 0 < len(block_code_documents):
            sub_indexes = False
            if 1 < len(block_code_documents):
                sub_indexes = True
            index = 1
            for code_doc in block_code_documents:
                if sub_indexes:
                    code_documents.append({
                        'index': block['id'],
                        'sub-index': index, 
                        'type': 'python',
                        'data': code_doc
                    })
                else:
                    code_documents.append({ 
                        'index': block['id'],
                        'sub-index': 0,
                        'type': 'python',
                        'data': code_doc
                    })
                index += 1
            
    formatted_documents = {
        'notebook-markdown': markdown_documents,
        'notebook-code': code_documents
    }
    
    return formatted_documents

In [76]:
example_notebook = pygithub_get_path_content(
    token = github_token,
    owner = 'K123AsJ0k1', 
    name = 'cloud-hpc-oss-mlops-platform', 
    path = 'tutorials/cloud_hpc/Cloud-HPC-FMNIST.ipynb'
)

In [97]:
notebook_documents = create_notebook_documents(
    notebook_text = example_notebook
)

In [68]:
def store_document(
    mongo_client: any,
    database_name: str,
    collection_name: str
):
    document_exists = mongo_check_collection(
        mongo_client = mongo_client, 
        database_name = database_name, 
        collection_name = collection_name
    )

    if not document_exists:
        


def store_github_repository_documents(
    repository_paths: any
) -> any:
    for path in repository_paths:
        path_split = path.split('/')
        document_database_name = path_split[-1].split('.')[-1]
        
        document_collection_name = ''
        for word in path_split[:-1]:
            document_collection_name += word[:2] + '|'
        document_collection_name += path_split[-1]

        
            

        #print(database_name)
        #print(collection_name)
        
        
        #print(storage_name)

In [ ]:
store_github_repository_documents(
    repository_paths = repository_paths
)

In [67]:
mongo_check_collection(
    mongo_client = mongo_client, 
    database_name = 'llm-rag-code', 
    collection_name = 'celery.py'
)

#mongo_get_document(
#    mongo_client = mongo_client, 
#    database_name = 'llm-rag-code', 
#    collection_name = 'celery.py', 
#    filter_query: any
#)

True